<a href="https://colab.research.google.com/github/danlingzhou16/stat390/blob/GraceZhu/Multiple_Prophet_All_Countries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import prophet
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
train = pd.read_csv('/content/gdrive/MyDrive/complete covid dataset/train_final.csv', parse_dates = ['date'])
test = pd.read_csv('/content/gdrive/MyDrive/complete covid dataset/test_final.csv', parse_dates = ['date'])

In [4]:
print(train.country_code.unique())


['US' 'LU' 'IE' 'NO' 'CH' 'SG' 'QA' 'IS' 'DK' 'AU']


In [5]:
from prophet.make_holidays import make_holidays_df

us_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='US')
us_holidays['country_code'] ='US'
lu_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='LU')
lu_holidays['country_code'] ='LU'
ie_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='IE')
ie_holidays['country_code'] ='IE'
no_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='NO')
no_holidays['country_code'] = 'NO'
ch_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='CH')
ch_holidays['country_code'] = 'CH'
sg_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='SG')
sg_holidays['country_code'] = 'SG'
# qa_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='QA') Qatar is not supported
is_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='IS')
is_holidays['country_code'] = 'IS'
dk_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='DK')
dk_holidays['country_code'] = 'DK'
au_holidays = make_holidays_df(year_list=[2020 + i for i in range(3)], country='AU')
au_holidays['country_code'] = 'AU'

In [7]:
holidays = pd.concat([us_holidays, lu_holidays, ie_holidays, no_holidays, ch_holidays, sg_holidays, is_holidays, dk_holidays, au_holidays])

In [9]:
holidays.holiday = 1
holidays.tail()

,ds,holiday,country_code
23,2022-01-03,1,AU
24,2022-04-15,1,AU
25,2022-04-18,1,AU
26,2022-12-25,1,AU
27,2022-12-27,1,AU


In [10]:
train_with_holiday = train.merge(holidays, how = 'left', left_on = ['date', 'country_code'], right_on = ['ds', 'country_code'])
train_with_holiday['holiday'].fillna(0, inplace = True)
test_with_holiday = test.merge(holidays, how = 'left', left_on = ['date', 'country_code'], right_on = ['ds', 'country_code'])
test_with_holiday['holiday'].fillna(0, inplace = True)

In [20]:
# drop useless columns
train_multi_prophet = train_with_holiday.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0', 'day_name', 'new_confirmed_mean1', 'new_confirmed_max1', 'new_confirmed_min1', 'ds', 'new_confirmed_std1'])
test_multi_prophet = test_with_holiday.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0', 'day_name', 'new_confirmed_mean1', 'new_confirmed_max1', 'new_confirmed_min1', 'ds', 'new_confirmed_std1'])

In [21]:
train_multi_prophet.head()

,date,location_key_x,country_code,new_deceased,cumulative_deceased,population,population_male,population_female,latitude,longitude,...,new_confirmed_min7,day_of_week,quarter,month,year,dayofyear,dayofmonth,weekofyear,season,holiday
0,2020-01-22,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,NaN,2,1,1,2020,22,22,4,Winter,0.0
1,2020-01-23,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,NaN,3,1,1,2020,23,23,4,Winter,0.0
2,2020-01-24,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,NaN,4,1,1,2020,24,24,4,Winter,0.0
3,2020-01-25,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,NaN,5,1,1,2020,25,25,4,Winter,0.0
4,2020-01-26,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,NaN,6,1,1,2020,26,26,4,Winter,0.0


In [22]:
# label encoding
from sklearn.preprocessing import LabelEncoder

label_encoder2 = LabelEncoder() # for season

# day of the week -- turns out there is a day_of_week column that has already been encoded
# however, Mon-Sun is from 0-6 and I want it to be 1-7
train_multi_prophet['day_of_week']= train_multi_prophet['day_of_week'] + 1
test_multi_prophet['day_of_week']= test_multi_prophet['day_of_week'] + 1
# season
train_multi_prophet['season'] = label_encoder2.fit_transform(train_multi_prophet['season'])
test_multi_prophet['season']= label_encoder2.transform(test_multi_prophet['season'])

In [23]:
# Group by countries
train_multi_prophet = train_multi_prophet.groupby(['country_code','date'])
test_multi_prophet = test_multi_prophet.groupby(['country_code','date'])

In [24]:
train_agg = train_multi_prophet.agg('sum')
train_agg.reset_index(level = 'country_code', inplace = True)
test_agg = test_multi_prophet.agg('sum')
test_agg.reset_index(level = 'country_code', inplace = True)

<ipython-input-24-b9667bd9aecf>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train_agg = train_multi_prophet.agg('sum')
<ipython-input-24-b9667bd9aecf>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_agg = test_multi_prophet.agg('sum')


In [26]:
train_agg

,country_code,new_deceased,cumulative_deceased,population,population_male,population_female,latitude,longitude,area_sq_km,life_expectancy,...,new_confirmed_min7,day_of_week,quarter,month,year,dayofyear,dayofmonth,weekofyear,season,holiday
date,,,,,,,,,,,,,,,,,,,,,
2020-01-27,AU,0.0,0.0,4668756.0,2013026.0,2022565.0,-76.000000,389.000000,4859125.0,247.20000,...,0.0,3,3,3,6060,81,81,15,6,3.0
2020-01-29,AU,0.0,0.0,4668756.0,2013026.0,2022565.0,-76.000000,389.000000,4859125.0,247.20000,...,0.0,9,3,3,6060,87,87,15,6,0.0
2020-01-30,AU,0.0,0.0,4668756.0,2013026.0,2022565.0,-76.000000,389.000000,4859125.0,247.20000,...,0.0,12,3,3,6060,90,90,15,6,0.0
2020-02-01,AU,0.0,0.0,4668756.0,2013026.0,2022565.0,-76.000000,389.000000,4859125.0,247.20000,...,0.0,18,3,6,6060,96,3,15,6,0.0
2020-02-04,AU,0.0,0.0,4668756.0,2013026.0,2022565.0,-76.000000,389.000000,4859125.0,247.20000,...,0.0,6,3,6,6060,105,12,18,6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,US,1742.0,817280.0,335073176.0,170796496.0,177855320.0,2073.424444,-4775.565556,9840963.0,4388.00661,...,19427.0,56,224,672,113176,20216,1512,2912,168,0.0
2021-12-28,US,2498.0,819778.0,335073176.0,170796496.0,177855320.0,2073.424444,-4775.565556,9840963.0,4388.00661,...,20785.0,112,224,672,113176,20272,1568,2912,168,0.0
2021-12-29,US,2100.0,821878.0,335073176.0,170796496.0,177855320.0,2073.424444,-4775.565556,9840963.0,4388.00661,...,20785.0,168,224,672,113176,20328,1624,2912,168,0.0


In [27]:
col_to_keep = ['country_code', 'day_of_week', 'quarter', 'month', 'year','dayofmonth','weekofyear','season','holiday']
addtional_col = train_agg.loc[:, train_agg.nunique() > 11].columns.to_list()
for col in addtional_col:
  col_to_keep.append(col)

In [28]:
col_to_keep

['country_code',
 'day_of_week',
 'quarter',
 'month',
 'year',
 'dayofmonth',
 'weekofyear',
 'season',
 'holiday',
 'new_deceased',
 'cumulative_deceased',
 'population',
 'population_male',
 'population_female',
 'latitude',
 'longitude',
 'area_sq_km',
 'life_expectancy',
 'mobility_workplaces',
 'aggregation_level',
 'new_persons_fully_vaccinated',
 'cumulative_persons_fully_vaccinated',
 'gdp_usd',
 'gdp_per_capita_usd',
 'AG.LND.AGRI.K2',
 'AG.LND.AGRI.ZS',
 'AG.LND.ARBL.HA',
 'AG.LND.ARBL.HA.PC',
 'AG.LND.ARBL.ZS',
 'AG.LND.CROP.ZS',
 'AG.LND.FRST.K2',
 'AG.LND.FRST.ZS',
 'AG.LND.TOTL.K2',
 'AG.PRD.CROP.XD',
 'AG.PRD.FOOD.XD',
 'AG.PRD.LVSK.XD',
 'AG.SRF.TOTL.K2',
 'BX.KLT.DINV.CD.WD',
 'EG.ELC.ACCS.RU.ZS',
 'EG.ELC.ACCS.UR.ZS',
 'EG.ELC.ACCS.ZS',
 'EG.ELC.RNEW.ZS',
 'EG.FEC.RNEW.ZS',
 'EN.ATM.CO2E.GF.KT',
 'EN.ATM.CO2E.GF.ZS',
 'EN.ATM.CO2E.KT',
 'EN.ATM.CO2E.LF.KT',
 'EN.ATM.CO2E.LF.ZS',
 'EN.ATM.CO2E.PC',
 'EN.ATM.CO2E.SF.KT',
 'EN.ATM.CO2E.SF.ZS',
 'EN.ATM.PM25.MC.M3',
 'EN

In [37]:
elements_to_remove = ['latitude', 'longitude', 'area_sq_km', 'life_expectancy', 'aggregation_level',
                      'mobility_workplaces', 'gdp_usd', 'gdp_per_capita_usd']

for element in elements_to_remove:
    if element in col_to_keep:
        col_to_keep.remove(element)

In [38]:
train_agg = train_agg[col_to_keep]
test_agg = test_agg[col_to_keep]

In [40]:

# put zeros to missing lagging features
train_agg.fillna(0, inplace = True)
test_agg.fillna(0, inplace = True)

<ipython-input-40-08689d7cd776>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_agg.fillna(0, inplace = True)


In [43]:
train_agg.head()

,country_code,day_of_week,quarter,month,year,dayofmonth,weekofyear,season,holiday,new_deceased,...,new_confirmed_min7,day_of_week,quarter,month,year,dayofyear,dayofmonth,weekofyear,season,holiday
date,,,,,,,,,,,,,,,,,,,,,
2020-01-27,AU,3,3,3,6060,81,15,6,3.0,0.0,...,0.0,3,3,3,6060,81,81,15,6,3.0
2020-01-29,AU,9,3,3,6060,87,15,6,0.0,0.0,...,0.0,9,3,3,6060,87,87,15,6,0.0
2020-01-30,AU,12,3,3,6060,90,15,6,0.0,0.0,...,0.0,12,3,3,6060,90,90,15,6,0.0
2020-02-01,AU,18,3,6,6060,3,15,6,0.0,0.0,...,0.0,18,3,6,6060,96,3,15,6,0.0
2020-02-04,AU,6,3,6,6060,12,18,6,0.0,0.0,...,0.0,6,3,6,6060,105,12,18,6,0.0


In [44]:
# Try US
train_US_multi = train_agg.loc[train_agg.country_code == 'US']
test_US_multi = test_agg.loc[test_agg.country_code == 'US']
train_US_multi.drop(columns = 'country_code', inplace = True)
test_US_multi.drop(columns = 'country_code', inplace = True)

<ipython-input-44-273e109db2c9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_US_multi.drop(columns = 'country_code', inplace = True)
<ipython-input-44-273e109db2c9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_US_multi.drop(columns = 'country_code', inplace = True)


In [56]:
train_US_multi.head()

,day_of_week,quarter,month,year,dayofmonth,weekofyear,season,holiday,new_deceased,cumulative_deceased,...,new_confirmed_min7,day_of_week,quarter,month,year,dayofyear,dayofmonth,weekofyear,season,holiday
date,,,,,,,,,,,,,,,,,,,,,
2020-01-13,1,1,1,2020,13,3,3,0.0,0.0,0.0,...,0.0,1,1,1,2020,13,13,3,3,0.0
2020-01-14,2,1,1,2020,14,3,3,0.0,0.0,0.0,...,0.0,2,1,1,2020,14,14,3,3,0.0
2020-01-15,3,1,1,2020,15,3,3,0.0,0.0,0.0,...,0.0,3,1,1,2020,15,15,3,3,0.0
2020-01-16,4,1,1,2020,16,3,3,0.0,0.0,0.0,...,0.0,4,1,1,2020,16,16,3,3,0.0
2020-01-17,5,1,1,2020,17,3,3,0.0,0.0,0.0,...,0.0,5,1,1,2020,17,17,3,3,0.0


In [57]:
train_US_multi.reset_index()

,date,day_of_week,quarter,month,year,dayofmonth,weekofyear,season,holiday,new_deceased,...,new_confirmed_min7,day_of_week,quarter,month,year,dayofyear,dayofmonth,weekofyear,season,holiday
0,2020-01-13,1,1,1,2020,13,3,3,0.0,0.0,...,0.0,1,1,1,2020,13,13,3,3,0.0
1,2020-01-14,2,1,1,2020,14,3,3,0.0,0.0,...,0.0,2,1,1,2020,14,14,3,3,0.0
2,2020-01-15,3,1,1,2020,15,3,3,0.0,0.0,...,0.0,3,1,1,2020,15,15,3,3,0.0
3,2020-01-16,4,1,1,2020,16,3,3,0.0,0.0,...,0.0,4,1,1,2020,16,16,3,3,0.0
4,2020-01-17,5,1,1,2020,17,3,3,0.0,0.0,...,0.0,5,1,1,2020,17,17,3,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,2021-12-27,56,224,672,113176,1512,2912,168,0.0,1742.0,...,19427.0,56,224,672,113176,20216,1512,2912,168,0.0
715,2021-12-28,112,224,672,113176,1568,2912,168,0.0,2498.0,...,20785.0,112,224,672,113176,20272,1568,2912,168,0.0
716,2021-12-29,168,224,672,113176,1624,2912,168,0.0,2100.0,...,20785.0,168,224,672,113176,20328,1624,2912,168,0.0
717,2021-12-30,224,224,672,113176,1680,2912,168,0.0,1416.0,...,20785.0,224,224,672,113176,20384,1680,2912,168,0.0


In [58]:
model2 = prophet.Prophet()
for col in train_US_multi.columns:
    if col not in ['date', 'new_confirmed']:
        model2.add_regressor(col)

In [61]:
x = train_US_multi.reset_index().rename(columns={'date':'ds', 'new_confirmed':'y'})

In [62]:
model2=model2.fit(x)

TypeError: ignored